In [1]:
import json
import altair as alt
from altair import expr, datum
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests

In [59]:
LOCAL = True

if LOCAL:
    local_suffix = "_local"
else:
    local_suffix = ""

In [60]:
%%capture pwd
!pwd

In [61]:
# uid = "2021-05-05-which-firms-and-industries-have-been-most-affected-by-covid-update"  # article unique ID
uid = pwd.stdout.split("/")[-1].split("\r")[0]
eco_git_home = (
    "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/"
)
eco_git_path = eco_git_home + "articles/" + uid + "/data/"
vega_embed = requests.get(eco_git_home + "guidelines/html/vega-embed.html").text
colors = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-colors.json").content
)
category_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-category-color.json").content
)
hue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-single-hue-color.json").content
)
mhue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-multi-hue-color.json").content
)
div_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-diverging-color.json").content
)
config = json.loads(
    requests.get(eco_git_home + "guidelines/charts/eco-global-config.json").content
)
height = config["height"]
width = config["width"]
uid, height, width

('2021-06-15-will-inflation-take-off-in-the-uk', 300, 500)

# Fig 1

In [62]:
df = pd.read_excel(
    "raw/Copy of Figure 18 - Inflation_FanChart.xlsx",
    sheet_name="UK HCP y-o-y",
    usecols="AE:AN",
    skiprows=1,
).dropna(how="all")
df.columns = ["date"] + ["c" + str(i) for i in df.columns[1:]]
df = df.dropna(subset=["date"])
df["date"] = [str(i) + "Q" + str(j) for i in range(2018, 2026) for j in range(1, 5)]
df = df.fillna(0)
df = df.set_index("date").astype(float).reset_index()

In [63]:
f = "fig1_inflation"
f1 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f1.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f1 = df
df.head()

,date,c10,c20,c30,c40,c50,c60,c70,c80,c90
0,2018Q1,2.6471,0.0,0.0,0.0,2.6471,0.0,0.0,0.0,0.0
1,2018Q2,2.4225,0.0,0.0,0.0,2.4225,0.0,0.0,0.0,0.0
2,2018Q3,2.5072,0.0,0.0,0.0,2.5072,0.0,0.0,0.0,0.0
3,2018Q4,2.1989,0.0,0.0,0.0,2.1989,0.0,0.0,0.0,0.0
4,2019Q1,1.9102,0.0,0.0,0.0,1.9102,0.0,0.0,0.0,0.0


In [126]:
base = alt.Chart(f1).encode(
    x=alt.X(
        "date:N",
        axis=alt.Axis(
            grid=False,
            title="",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            orient="bottom",
            # labelExpr="indexof(datum.value,'Q1')>-1?slice(datum.value,0,4)+' '+slice(datum.value,4):slice(datum.value,4)",
            labelExpr="indexof(datum.value,'Q1')>-1?slice(datum.value,0,4):''",
            labelAngle=0,
        ),
    )
)
line = base.mark_line(color=colors["eco-background"]).encode(
    y=alt.Y(
        "c50:Q",
        title="",
        axis=alt.Axis(
            grid=True,
            title="%",
            titleAnchor="start",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            titleFontSize=10,
            titleFontWeight="normal",
            ticks=False,
            labelAlign="left",
            labelBaseline="middle",
            labelPadding=-5,
            labelOffset=-10,
            titleX=15,
            titleY=-5,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            tickCount=7,
            format=".0f",
        ),
    )
)
area1 = (
    base.mark_area(color=colors["eco-turquiose"], opacity=0.5)
    .encode(y=alt.Y("c10:Q"), y2=alt.Y2("c21:Q"))
    .transform_calculate(c21="datum.c10+datum.c20")
)
area2 = (
    area1.mark_area(color=colors["eco-turquiose"], opacity=0.8)
    .encode(y=alt.Y("c21:Q"), y2=alt.Y2("c31:Q"))
    .transform_calculate(c31="datum.c21+datum.c30")
)
area3 = (
    area2.mark_area(color=colors["eco-light-blue"], opacity=0.8)
    .encode(y=alt.Y("c31:Q"), y2=alt.Y2("c41:Q"))
    .transform_calculate(c41="datum.c31+datum.c40")
)
area4 = (
    area3.mark_area(color=colors["eco-mid-blue"], opacity=0.8)
    .encode(y=alt.Y("c41:Q"), y2=alt.Y2("c61:Q"))
    .transform_calculate(c61="datum.c41+datum.c60")
)
area5 = (
    area4.mark_area(color=colors["eco-light-blue"], opacity=0.8)
    .encode(y=alt.Y("c61:Q"), y2=alt.Y2("c71:Q"))
    .transform_calculate(c71="datum.c61+datum.c70")
)
area6 = (
    area5.mark_area(color=colors["eco-turquiose"], opacity=0.8)
    .encode(y=alt.Y("c71:Q"), y2=alt.Y2("c81:Q"))
    .transform_calculate(c81="datum.c71+datum.c80")
)
area7 = (
    area6.mark_area(color=colors["eco-turquiose"], opacity=0.5)
    .encode(y=alt.Y("c81:Q"), y2=alt.Y2("c91:Q"))
    .transform_calculate(c91="datum.c81+datum.c90")
)
xAxis = (
    alt.Chart(pd.DataFrame([{"value": 0}]))
    .mark_rule(size=0.5)
    .encode(
        y=alt.Y("value:Q"),
    )
)
layer = (
    (xAxis + area1 + area2 + area3 + area4 + area5 + area6 + area7 + line)
    .configure_view(stroke=None)
    .properties(title="")
    .properties(height=300, width=500)
)
layer.save("visualisation/" + f + ".json")
layer

alt.LayerChart(...)